<a href="https://colab.research.google.com/github/thaonguyyen/DS2002-HW/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thao Nguyen (vdk4dy)

Matthew Haid (wsd6vn)

Larissa Cybyk (jdj9nj)

Import necessary libraries:

In [82]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np
import requests

In [83]:
def get_weather_forecast(city_name):
    # Tomorrow.io API URL and key
    api_key = 'c4zmWjxrILFRN50mkGeSIQ71MPJhJjuq'
    url = f'https://api.tomorrow.io/v4/weather/forecast?location={city_name}&apikey={api_key}'

    # Make the GET request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        return data["timelines"]['hourly'][0]
    else:
        return f'Failed to get weather forecast. Status code: {response.status_code}'

Initialize chatbot:

In [84]:
questions = [
    'Hello.',
    'Hey',
    'Hi',
    'How are you?',
    'What is your name?',
    'Bye',
    'Goodbye',
    'Later',
    'What is the weather?'
]

answers = [
    'Hello.',
    'Hey!',
    'Hi!',
    'I am just a bot, but I am doing fine.',
    'I am a chatbot created by OpenAI.',
    'Bye!',
    'Goodbye. See you later!',
    'See you later!',
    'Let me check the weather.'
]

Tokenize the questions and answers:

In [85]:
# tokenize the sentences
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_questions = tokenizer.texts_to_sequences(questions)
sequences_answers = tokenizer.texts_to_sequences(answers)

# pad the sequences
max_length = max(max(len(seq) for seq in sequences_questions), max(len(seq) for seq in sequences_answers))
padded_questions = tf.keras.preprocessing.sequence.pad_sequences(sequences_questions, maxlen=max_length, padding='post')
padded_answers = tf.keras.preprocessing.sequence.pad_sequences(sequences_answers, maxlen=max_length, padding='post')

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

Define model:

In [86]:
# define model parameters
embedding_dim = 256
units = 1024

# define the encoder model
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# define the decoder model
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# define the seq2seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# summary of the model
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_23 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 input_24 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_22 (Embedding)    (None, None, 256)            8192      ['input_23[0][0]']            
                                                                                                  
 embedding_23 (Embedding)    (None, None, 256)            8192      ['input_24[0][0]']            
                                                                                           

Prepare decoder and train model:

In [87]:
# prepare decoder input data that just contains the start token and the rest is all 0
decoder_input_data = np.zeros_like(padded_answers)
decoder_input_data[:, 0] = 1  # assuming 1 is the start token

# train the model
model.fit([padded_questions, decoder_input_data], np.expand_dims(padded_answers, -1), batch_size=2, epochs=100)

Epoch 1/100
5/5 [==============================] - 4s 54ms/step - loss: 3.1100 - accuracy: 0.4778
Epoch 2/100
5/5 [==============================] - 0s 31ms/step - loss: 1.7688 - accuracy: 0.6333
Epoch 3/100
5/5 [==============================] - 0s 29ms/step - loss: 1.8425 - accuracy: 0.6333
Epoch 4/100
5/5 [==============================] - 0s 14ms/step - loss: 2.3454 - accuracy: 0.6333
Epoch 5/100
5/5 [==============================] - 0s 28ms/step - loss: 1.8448 - accuracy: 0.6333
Epoch 6/100
5/5 [==============================] - 0s 14ms/step - loss: 1.6292 - accuracy: 0.6333
Epoch 7/100
5/5 [==============================] - 0s 13ms/step - loss: 1.4073 - accuracy: 0.6222
Epoch 8/100
5/5 [==============================] - 0s 14ms/step - loss: 1.7817 - accuracy: 0.6333
Epoch 9/100
5/5 [==============================] - 0s 13ms/step - loss: 1.6707 - accuracy: 0.6333
Epoch 10/100
5/5 [==============================] - 0s 13ms/step - loss: 1.6245 - accuracy: 0.6333
Epoch 11/100
5/5 [=

Implement chat methods:

In [88]:
def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1  # start token
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:  # end token
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

def chat_with_bot(input_text):
    input_sequence = preprocess_input_text(input_text)
    response_sequence = generate_response(input_sequence)
    response_text = sequence_to_text(response_sequence[0])
    return response_text

Interactive chat:

In [89]:
# Interactive chat with the bot
print("Start chatting with the bot! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
      print("Goodbye, thanks for chatting!")
      break
    response = chat_with_bot(user_input)
    print(f"Bot: {response}")
    if user_input.find("weather") >= 0:
      city = input("Bot: What city would you like to check the weather for: ")
      json = get_weather_forecast(city)
      temp = json['values']['temperature']
      uv = json['values']['uvIndex']
      print(f"Bot: {city} has a temperature of {temp} degrees Celsius and a UV index of {uv}.")

Start chatting with the bot! Type 'quit' to exit.
You: quit
Goodbye, thanks for chatting!
